### Importing all the necessary modules for the application.

__*IMPORTANT:* After installing the packages, please restart the Jupyter kernal for the changes to take place.__

In [1]:
import pkgutil

# installing Diffusers package for Stable Diffusion Image Generation.
if pkgutil.find_loader('diffusers') is None:
    %pip install diffusers
else:
    print('diffusers already installed')

# installing PyTorch for Transformers, accelerators and SafeTensors.
if pkgutil.find_loader('torch') is None:
    %pip install torch
else:
    print('torch already installed')

if pkgutil.find_loader('torchvision') is None:
    %pip install torchvision
else:
    print('torchvision already installed')

# installing the Invisible WaterMarking package.
if pkgutil.find_loader('invisible_watermark') is None:
    %pip install invisible_watermark
else:
    print('invisible_watermark already installed')

# installing Gradio for the GUI.
if pkgutil.find_loader('gradio') is None:
    %pip install gradio
else:
    print('gradio already installed')

# installing transformers
if pkgutil.find_loader('transformers') is None:
    %pip install transformers
else:
    print('transformers already installed')

# installing Accelerate
if pkgutil.find_loader('accelerate') is None:
    %pip install accelerate
else:
    print('accelerate already installed')

# installing SafeTensors
if pkgutil.find_loader('safetensors') is None:
    %pip install safetensors
else:
    print('safetensors already installed')

# !pip install torch torchvision cudatoolkit=12.2

# importing packages
import torch
from diffusers import DiffusionPipeline
import gradio as gr



diffusers already installed
torch already installed
torchvision already installed
Note: you may need to restart the kernel to use updated packages.
gradio already installed
transformers already installed
accelerate already installed
safetensors already installed


In [4]:
import torch

print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.0.1+cpu
Is CUDA enabled? False


### Loading the "Base" and "Refiner" models of Stable Diffusion XL

we use the *Base* model to generate the image, and use *Refiner* model to refine the generated image by applying post-processing effects, upscalling, etc.

In [3]:
# loading the Base model
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype = torch.float16,
    variant = "fp16",
    use_safetensors = True
)
base.to("cuda")

# loading the Refiner model
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2 = base.text_encoder_2,
    vae = base.vae,
    torch_dtype = torch.float16,
    variant = "fp16",
    use_safetensors = True
)
refiner.to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

AssertionError: Torch not compiled with CUDA enabled

### Loading the "Upscaler" model

As the name suggests, it upscales or improves the resolution of the generated images.
This is needed as these models typically support very low resolutions (in this case, a resolution of upto 1024x1024px) which might not be very sharp for sharing your AI masterpieces online.

In [ ]:
# loading the upscaler
upscaler = DiffusionPipeline.from_pretrained(
    "stabilityai/sd-x2-latent-upscaler",
    torch_dtype=torch.float16,
    use_safetensors=True
)
upscaler.to("cuda")

### Defining a function that handles the image generation process

We pass in the prompt, image heigth, and width that the user gives, which is then sent to the models for generating the image.

In [ ]:
def image_generator(prompt, negative_prompt, img_height, img_width, 
                    guidance, steps, seed, enable_upscaling, high_noise_fraction):

    generator = torch.Generator(device = "cuda").manual_seed(seed)
    init_image = base(
        prompt = prompt,
        negative_prompt = negative_prompt,
        height = img_height,
        width = img_width,
        guidance_scale = guidance,
        num_inference_steps = steps,
        generator = generator,
        denoising_end = high_noise_fraction,
    ).images

    if enable_upscaling:
        final_image = refiner(
            
        )
        


In [1]:
%pip uninstall torch

^C
Note: you may need to restart the kernel to use updated packages.
